In [23]:
import time
import copy
import os

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
DATA_ROOT = 'C:/Users/chira/Desktop/VideoAnalysisTry/screenshots/Cashier'


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), # Augmentation
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
}


train_dataset = datasets.ImageFolder(root=DATA_ROOT + '/Train', transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root=DATA_ROOT + '/Test', transform=data_transforms['val'])

# Determine the number of classes from the loaded dataset
num_classes = len(train_dataset.classes)
BATCH_SIZE = 32

# 2. Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [24]:
num_classes = len(train_dataset.classes)
BATCH_SIZE = 32
num_workers = 0 # Set to 0 if encountering issues, otherwise use 4 or more

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

dataloaders = {'train': train_loader, 'val': test_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(test_dataset)}


# --- 2. RESNET MODEL SETUP ---
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Number of classes detected: {num_classes}")

# Load pre-trained ResNet-50
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Freeze all parameters in the convolutional base (Feature Extractor strategy)
for param in model.parameters():
    param.requires_grad = False

# Get the number of input features to the final classification layer
num_ftrs = model.fc.in_features

# Replace the final fully connected layer to match your num_classes
model.fc = nn.Linear(num_ftrs, num_classes)

model = model.to(device)

# --- 3. OPTIMIZER AND LOSS ---
criterion = nn.CrossEntropyLoss()
# Optimizer only updates the parameters whose requires_grad=True (i.e., the new fc layer)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Learning Rate Scheduler (optional but good practice)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


# --- 4. TRAINING FUNCTION ---
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 20)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass: track history only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step() # Update learning rate

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it's the best validation accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# --- 5. EXECUTE TRAINING ---
# Set the number of epochs you want to train for
NUM_EPOCHS = 15

# Start training!
print("\n--- Starting ResNet Fine-Tuning ---")
final_model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=NUM_EPOCHS)

# --- 6. SAVE THE MODEL ---
torch.save(final_model.state_dict(), 'resnet50_final_weights.pth')
print("\nModel weights saved to resnet50_final_weights.pth")

Using device: cpu
Number of classes detected: 2

--- Starting ResNet Fine-Tuning ---
Epoch 1/15
--------------------
Train Loss: 0.5239 Acc: 0.8213
Val Loss: 0.4062 Acc: 0.8440
Epoch 2/15
--------------------
Train Loss: 0.4428 Acc: 0.8399
Val Loss: 0.4783 Acc: 0.8440
Epoch 3/15
--------------------
Train Loss: 0.4384 Acc: 0.8399
Val Loss: 0.4320 Acc: 0.8440
Epoch 4/15
--------------------
Train Loss: 0.4388 Acc: 0.8399
Val Loss: 0.4313 Acc: 0.8440
Epoch 5/15
--------------------
Train Loss: 0.4141 Acc: 0.8399
Val Loss: 0.4246 Acc: 0.8440
Epoch 6/15
--------------------
Train Loss: 0.4267 Acc: 0.8399
Val Loss: 0.4213 Acc: 0.8440
Epoch 7/15
--------------------
Train Loss: 0.4281 Acc: 0.8399
Val Loss: 0.4250 Acc: 0.8440
Epoch 8/15
--------------------
Train Loss: 0.4234 Acc: 0.8399
Val Loss: 0.4215 Acc: 0.8440
Epoch 9/15
--------------------
Train Loss: 0.4046 Acc: 0.8399
Val Loss: 0.4217 Acc: 0.8440
Epoch 10/15
--------------------
Train Loss: 0.4143 Acc: 0.8399
Val Loss: 0.4182 Acc: 0